# **Using LLM assitance**

If we want a lightweigth model that can generate Shakespearian sonnets, we realized that the model by itself is challenging. While it manages to get the goals we had set up for the train and test set, its ability to create generalized structure is limited.

However, this open an opportunity to incorporate the thing that everyone is talking about (but without just asking it to solve a task for us and trusting the black box): LLMs!

## How can we do it?

Instead of asking the LLM to generate a sonnet, we can provide it with the generated sonnet from our model and ask it to correct it. Eliminate duplicate words, strange structures, and assist line breaking. In this way, we can be more in control of the LLM, asking it to not alter our original sonnet, but do what it does best: work with patterns. Because, in the end of the day, a model like Gemini probably already "read" all sonnets written by humanity...

In [1]:
import os
from dotenv import load_dotenv
import google.generativeai as genai
import torch

from model import TransformerModel
from utils import generate_sonnet_sampling, clean_generated_text

import sentencepiece as spm

load_dotenv()

GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')

/home/joaolucasmbc/mambaforge/envs/nlp-shakespeare/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(f"Using device: {device}")

Using device: cuda


In [3]:
model = TransformerModel(
    vocab_size=4000, 
    seq_len=256, 
    embedding_dim=256, 
    num_heads=8, 
    num_layers=6, 
    dropout=0.1
).to(device)

model_path = 'model/checkpoint_epoch_200_valloss_1.9365.pth'

# Load
if os.path.exists(model_path):
    model.load_state_dict(torch.load(model_path, map_location='cpu'))
    print(f"Model loaded from {model_path}")

Model loaded from model/checkpoint_epoch_200_valloss_1.9365.pth


/tmp/ipykernel_90387/2830344480.py:14: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model_path, map_location='cpu'))


The distribution that yielded some of the better results while reading was using sampling.

In [4]:
sp = spm.SentencePieceProcessor(model_file='tokenizer/my_tokenizer.model')

In [5]:
model.eval()

start_text = "Thy"
raw = generate_sonnet_sampling(model, sp, start_text, max_length=256, device='cuda')
generated_sonnet = clean_generated_text(raw, pad_token='<PAD>')
print(generated_sonnet)

Thy both most ensnared our honor our honor honor friendship respect lies read lies in our being- our love forever lives lies our thee to the honor witht th love stoneive learn to discern core within through life by labory itst phrases in its sacred She ourt
 Yet death to plague he belong me reside, So simple's deceptive dost black lies, Over granted scholars by its amongmp find believ gaze find its decreeyis skillownys worth outshine free. 
 Ens fail peace no more our destiny entwined in ourselves inv haless repent, ourselves concealed within our age no stage lead; 
 Seducing our artist in this ground extreme; 
 Speak restmeri solace glare, our secret, with its profan bloom that flourishes both grief maintain thus, better tainteds vast archive shall kindly Nature'll needs're fashion has madey expression to move by tears and moonlit air.


## Creating a prompt

In [6]:
# Start the use of the API
genai.configure(api_key=GEMINI_API_KEY)

# Make our prompt here
prompt = f"""
You are an expert and a teacher in Shakespearing sonnets. Your goal is to take your student's sonnet and help him polish it. You need to pay attention to:
* Duplicate words
* Repetitive phrases
* Line breaks (always has to be a sonnet)
* Slight mispells
* Lack of rhymes
* Words that were clearly stuck together

You are NOT allowed to change the student's words or their meaning, that will make him sad!

Your output should be the new sonnet. No explanations, no anything more or less.

Here is the student's sonnet:
{generated_sonnet}
"""

generation_config = genai.GenerationConfig(
    max_output_tokens=256,
    temperature=1.0,
)

# Use our prompt 
model = genai.GenerativeModel(model_name="gemini-1.5-flash")

response = model.generate_content(prompt,
                                  generation_config=generation_config)

print("Response from Gemini:")
print()
print(response.candidates[0].content.parts[0].text)

Response from Gemini:

Thy ensnared honor, friendship, respect, lies read
In our being; our love forever lives, lies
In thee; the honor, love, stone I've learned to discern,
Core within, through life's labor, its phrases, sacred.

Yet death may plague; he belongs, resides in me.
So simple's deceptive, black lies,
Over-granted scholars find, believe, gaze, find
Its decree; its skill, own worth outshines free.

Ensnare peace; no more our destiny entwined,
In ourselves, unless repent; ourselves concealed
Within our age, no stage leads;
Seducing our art in this extreme ground.

Speak, rest; mere solace glares; our secret,
With its profane bloom, flourishes; both grief
Maintained, vast archive shall kindly nature's fashion
Express; to move by tears and moonlit air.



In [26]:
print(generated_sonnet)

Thy their known ruthless lament their their these muse reveal me vain imposed history's of sightcauldronion from thee. figure first within the these unwritten my of Decoright from the absent, did speak toos. 
. akatch use is from the upon myju fall their violet I baseamand reason light of monarchs bold to fall away lies wanting ardor gulf naught 
 Nor Marsing and bright Pardoning bow de sight lent. 
 Thus on Death bow to nature call, these lines, gracing her head richness 
 For grow from those pe Ardor of grasp of hollow glory, adorned confess my feeble they have victor fate my far from their guest, claim I think beneath the sun'll declare; qu 
 Here song amidst grace flares gates. 
 Why fearch thrives, seek treasure voy and remorselessiness die of stars lost, Shall fade, yearning totholo doessers, reviled.


# Creating 3 sonnets

Prompts always have 8 words to start. 

All sonnets came from [this data source](https://www.readwritethink.org/sites/default/files/resources/lesson_images/lesson830/sonnet-links.html). The first 3 were used in this test.

In [5]:
# Start the use of the API
genai.configure(api_key=GEMINI_API_KEY)

# Make our prompt here
prompt = f"""
You are an expert and a teacher in Shakespearing sonnets. Your goal is to take your student's sonnet and help him polish it. You need to pay attention to:
* Duplicate words
* Repetitive phrases
* Line breaks (always has to be a sonnet)
* Slight mispells
* Lack of rhymes
* Words that were clearly stuck together

You are NOT allowed to change the student's words or their meaning, that will make him sad!

Your output should be the new sonnet. No explanations, no anything more or less.

Here is the student's sonnet: 
"""

generation_config = genai.GenerationConfig(
    max_output_tokens=256,
    temperature=1.0,
)

# Use our prompt 
llm = genai.GenerativeModel(model_name="gemini-1.5-flash")

In [11]:
model.eval()

start_text = "Earth has not anything to show more fair:"
raw = generate_sonnet_sampling(model, sp, start_text, max_length=256, device='cuda')
generated_sonnet = clean_generated_text(raw, pad_token='<PAD>')
print(generated_sonnet)

Earth has not anything to show more fair: flattery find respect for  to multitudellinting disdain 
 Bright and beauty, yet green lost Unfair to common to grow to the tales  too, remain, clear by errors isolationlain. 
 veins Sings zenithing, fairly the chains guise, defy shatterly speak provoke for my rem,aning eclipses increase by reprieve, unfa Time run thy devotion increase, has captures to a shadow graced by Time's fades away free. blely winter cannot straying rotlit grace one thou shalt Death wouldfere falls to grayears would with ease. 
 Feed. looms death to Reviv eternal pride, extremeived toose it pleasess accord? ve lost where life may cease. 
 Who granted have sworn whispers both to find dark ve'tis cause creature.


In [13]:
response = llm.generate_content(prompt + generated_sonnet,
                                  generation_config=generation_config)

print("Response from Gemini:")
print()
print(response.candidates[0].content.parts[0].text)

Response from Gemini:

Earth has not anything to show more fair:
Flattery finds respect for multitudinous disdain.
Bright beauty, yet green lost, unfair, common to grow.
The tales remain, clear by errors, isolation lain.

Veins sing zenithing, fairly the chains guise.
Defy shatteringly; speak, provoke my remaining eclipses.
Increase by reprieve, unfaded time runs.
Thy devotion increases; captures a shadow graced by time's fades away free.

Bleak winter cannot stray; rotting grace, one thou shalt.
Death would prefer falls to gray years with ease.
Feed. Death looms to revive eternal pride, extreme.
It pleases; accords? We've lost where life may cease.

Who granted have sworn whispers both to find.
Dark veils; it is cause; creature.



In [14]:
model.eval()

start_text = "Death, be not proud, though some have called"
raw = generate_sonnet_sampling(model, sp, start_text, max_length=256, device='cuda')
generated_sonnet = clean_generated_text(raw, pad_token='<PAD>')
print(generated_sonnet)

Death, be not proud, though some have called ; arnished by these unwritten from with scroll- Witness, xict the not, from their devoid concern piece for is solace, "In its Se him wellspringed prayer far understand outwardnoc of time has consume these lines on both pen utter eye grow and ardor Slowly come to truth' hands inspire their words on fields of blameistend bidnor march of burden? 
 This in thus: of imprint of self is so bright when beautiful sight, and found in chest; 
 Ardor the tale of time had oneadeoroake. 
 Whoso restrains? st weed, though. 
 Through is not rise eyes; don, lent or earthing wisdom from me defiant? ephemeral? wo disarray can hide we, all has flower I cannot restore nores compel vice is through duty? intoed honor anthology.


In [15]:
response = llm.generate_content(prompt + generated_sonnet,
                                  generation_config=generation_config)

print("Response from Gemini:")
print()
print(response.candidates[0].content.parts[0].text)

Response from Gemini:

Death, be not proud, though some have called thee
adorned by these unwritten scrolls —
Witness not, from their devoid concern,
a solace, "In its wellspring," prayer, far understand
outward reach of time. Consume these lines;
Both pen, and eye, grow old, and ardor
slowly comes to truth. Hands inspire
their words on fields of blame.  Attend; nor march of burden?

This imprint of self is so bright,
when beautiful, a sight, and found in the chest;
Ardor, the tale of time, had oneadeoroake.
Whoso restrains?  A weed, though.

Through eyes, not risen, lent or earthing wisdom;
Defiant? Ephemeral? Woe! Disarray
cannot hide. We, all, have flowered.
I cannot restore, nor compel. Vice is through duty? Intoed honor, anthology.



In [17]:
model.eval()

start_text = "Death, be not proud, How do I love thee? Let me count "
raw = generate_sonnet_sampling(model, sp, start_text, max_length=256, device='cuda')
generated_sonnet = clean_generated_text(raw, pad_token='<PAD>')
print(generated_sonnet)

Death, be not proud, How do I love thee? Let me count I be path without me hence parts love should make so smooth own then, 
 Letified To keep wisdom'er was self, me serve from myself, thoughere wiltedmplayed How, I could fall now tainted, with care. 
 Con veil I fulfill your nimble me with tenschool judge back solveestified torment thank lack to self, swiftly trace. worms beyond compare? By lips and love from ills deception sings in chastedors, held in thy freedom both dark veil in air, be true lingers weep me can join. 
 To keep and broken on, though defam? Would both, abandon my beloved bow? 
 It last. 
 Abandoned and come by actions tainted, when you both strength beyond held, love forever lay: merit. 
 Passion, dost and its faults, thus, in?


In [20]:
response = llm.generate_content(prompt + generated_sonnet,
                                  generation_config=generation_config)

print("Response from Gemini:")
print()
print(response.candidates[0].content.parts[0].text)

Response from Gemini:

Death, be not proud. How do I love thee? Let me count.
I be path without me. Hence, parts love should make so smooth.
Own then, letified to keep wisdom. 'Twas self, me serve from myself.
Though ere wilted, I played. How I could fall, now tainted, with care.
Con veil I fulfill. Your nimble me, with ten, school judge.
Back, solve, letified torment. Thank lack to self, swiftly trace.
Worms beyond compare? By lips and love from ills, deception sings.
In chaste doors, held in thy freedom. Both dark veil in air, be true.
Lingers. Weep. Me can join. To keep and broken on, though defamed?
Would both abandon my beloved bow? It last.
Abandoned, and come by actions tainted. When you both strength beyond held,
Love forever lay: merit. Passion, dost and its faults, thus, in?

